# HW4: QA Agent

## Dependencies and LLM Backbone

In [1]:
# !pip install langchain==1.0.5
# !pip install langchain-core
# !pip install langchain-community
# !pip install faiss-cpu
# !pip install kagglehub
device = "cuda"  # "cpu" or "cuda"

In [2]:
# This is the list of countries we are using (with their official languages)
# Feel free to use it in your code
list_of_countries = {}
with open("countries_with_languages.tsv", "r"  ) as f:
    for line in f.readlines():
        country, langs = line.strip().split("\t")
        list_of_countries[country] = langs.split(",")

### Choice 1: OpenAI API

The notebook's implementation is based on this.
Feel free to change the model, and please keep track of your usage on the "Usage" page on [LiteLLM API webpage](https://ai-gateway.andrew.cmu.edu/ui/).

In [3]:
# !pip install langchain-openai

In [4]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
import getpass, os

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API Key: ")
openai_model_id = "gpt-5"
openai_embmodel_id = "azure/text-embedding-3-small"

llm = ChatOpenAI(
    model=openai_model_id,
    api_key=os.environ["OPENAI_API_KEY"],
    base_url="https://ai-gateway.andrew.cmu.edu/"
)
embeddings = OpenAIEmbeddings(
    model=openai_embmodel_id,
    api_key=os.environ['OPENAI_API_KEY'],
    base_url='https://ai-gateway.andrew.cmu.edu/'
)

### Choice 2: Hugging Face Models

You may also use Hugging Face models without API credits if you have available GPU resource. You might have to the change prompt templates according to your model choice.

In [5]:
# !pip install langchain-huggingface text-generation transformers google-search-results 
# !pip install numexpr langchainhub sentencepiece sentence-transformers jinja2 bitsandbytes accelerate

In [6]:
import getpass, os, torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_huggingface import ChatHuggingFace, HuggingFacePipeline, HuggingFaceEmbeddings

os.environ["HUGGINGFACEHUB_API_TOKEN"] = getpass.getpass("Enter your Hugging Face API key: ")
hgf_model_id = "Qwen/Qwen3-0.6B"
hgf_embmodel_id = "sentence-transformers/all-mpnet-base-v2"

# Select device and dtype
device = "cuda" if torch.cuda.is_available() else "cpu"
dtype = torch.float16 if device == "cuda" else torch.float32

# Build HF generation pipeline on GPU when available
tokenizer = AutoTokenizer.from_pretrained(hgf_model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    hgf_model_id,
    torch_dtype=dtype,
    device_map="auto",
    trust_remote_code=True,
)

gen_pipe = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto",
    torch_dtype=dtype,
)

hgf_model = HuggingFacePipeline(pipeline=gen_pipe)
hgf_llm = ChatHuggingFace(llm=hgf_model)

# Place sentence-transformers embeddings on GPU when available
hgf_embeddings = HuggingFaceEmbeddings(
    model_name=hgf_embmodel_id,
    model_kwargs={"device": device}
)

2025-12-03 08:36:23.968740: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764750983.997946   63645 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764750984.010012   63645 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-12-03 08:36:24.258471: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
`torch_dtype` is deprecated! Use `dtype` instead!
`torch_dtype` is deprecated! Use `dtype` instead!
Device 

## Handling different type of questions

Implement the answer formatting and extraction for each question type. You may change the prompt to fit your processing function.

In [7]:
from langchain.agents import create_agent

### 🗺️Global Trekker

In [8]:
global_trekker_messages = [
    {"role": "system", "content": "You are an expert in world geography and cultural geography. Given a descriptive paragraph, infer the most likely country and city with high precision."},
    {"role": "user", "content": """
Read the paragraph and extract:
- Country: use the full official country name (e.g., "United States", not "USA").
- City: a specific city if clearly indicated by clues (landmarks, neighborhoods, transit lines, local foods, dialects). If not identifiable, write "Unknown".

Strict output format (no explanations):
[Country], [City]

Guidelines:
- Prefer unique cues (local transit names, street names, postal formats, phone codes, currency, cuisine, sports clubs).
- If multiple cities fit, pick the single most likely one.
- If the paragraph is generic or only country-level, return "Unknown" for city.
- Do not include any extra text besides the bracketed pair.

Examples:
Input: "We walked along the Arno past the Ponte Vecchio before climbing to Piazzale Michelangelo."
Output: [Italy], [Florence]

Input: "A red double-decker bus passed by the Thames near Westminster Abbey and Big Ben."
Output: [United Kingdom], [London]

Input: "We grabbed Primanti's near the confluence of the Allegheny and Monongahela."
Output: [United States], [Pittsburgh]

Input: "I toured the Old Town and the Royal Mile before seeing the castle on the hill."
Output: [United Kingdom], [Edinburgh]

Input: "We enjoyed maple syrup and poutine while skating on a canal in winter."
Output: [Canada], [Ottawa]
"""},
]

global_trekker = create_agent(model=llm, tools=[])

In [9]:
def extract_global_trekker_answer(response: str) -> tuple[str, str]:
    # TODO: Extract the country and city from the response
    # return country, city
    import re
    
    # Look for pattern [country], [city] or variations
    # Try to find content within brackets first
    bracket_pattern = r'\[([^\]]+)\]\s*,\s*\[([^\]]+)\]'
    match = re.search(bracket_pattern, response)
    
    if match:
        country = match.group(1).strip()
        city = match.group(2).strip()
        return country, city
    
    # Fallback: look for two items separated by comma
    # Often LLMs will say something like "United States, Pittsburgh"
    lines = response.strip().split('\n')
    for line in lines:
        # Skip lines that are too long (likely explanations)
        if len(line) > 100:
            continue
        # Look for comma-separated values
        if ',' in line:
            parts = line.split(',')
            if len(parts) >= 2:
                country = parts[0].strip().strip('[]"\'')
                city = parts[1].strip().strip('[]"\'')
                # Clean up common prefixes
                for prefix in ['The answer is', 'Answer:', 'Location:', 'Country:', 'City:']:
                    country = country.replace(prefix, '').strip()
                    city = city.replace(prefix, '').strip()
                return country, city
    
    # Last resort: return empty strings
    return "", ""

In [10]:
# Test run your extration function before using it in the main loop!
extract_global_trekker_answer("AAA, BBB")

('AAA', 'BBB')

### 🍽️Culinary Detective

In [11]:
import kagglehub
import pandas as pd

def gather_recipe_data(kaggledataset: str) -> list[str]:
    dataset_path = kagglehub.dataset_download(kaggledataset)
    df = pd.read_csv(f"{dataset_path}/Receipes from around the world.csv", encoding='latin-1')
    
    # Process the dataframe to list of text entries for retrieval
    # Format each recipe as structured text for better retrieval
    recipes = []
    for _, row in df.iterrows():
        # Create a readable text representation of each recipe
        recipe_parts = []
        for col in df.columns:
            value = row[col]
            # Skip NaN values and format nicely
            if pd.notna(value) and str(value).strip():
                recipe_parts.append(f"{col}: {value}")
        
        # Join all parts into a single text entry
        recipe_text = ". ".join(recipe_parts)
        recipes.append(recipe_text)
    
    return recipes

In [12]:
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langchain.tools import tool

recipes = gather_recipe_data("prajwaldongre/collection-of-recipes-around-the-world")
docs = [Document(page_content=recipe) for recipe in recipes]
vector = FAISS.from_documents(docs, embeddings)
retriever = vector.as_retriever(search_kwargs={"k": 2})

## RAG Tool
I created the following:
- a folder to store embeddings and faiss index
- a rag pipeline file
- a file that exposes the rag pipeline as a tool. 
- I am importing the tool in here. 

In [13]:
from langchain.tools import tool
from rag_system.rag_pipeline import CulinaryRAG

# Load the RAG engine and retriever
rag = CulinaryRAG()
retriever = rag.load_index()  # returns vectorstore.as_retriever()

@tool
def retrieve_culinary_context(query: str):
    """
    Retrieves culinary information relevant to country/region origin detection.
    Takes a descriptive query (ingredients, cooking method, spices) and performs vector retrieval.
    """
    docs = retriever.invoke(query)
    return "\n".join([d.page_content for d in docs])

/home/sagemaker-user/codespace/nlp_assignments/qa_agent/rag_system/rag_pipeline.py:13: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  self.embeddings = HuggingFaceEmbeddings(


In [14]:
@tool
def retrieve_recipes(query: str):
  """
  Retrieves recipes based on a search query.
  """
  return retriever.invoke(query)

culinary_detective_messages = [
    {"role": "system", "content": """
You are an expert culinary anthropologist. From ingredients and a short description, identify the country and the SPECIFIC region within that country where the dish is most associated.

You can consult a retrieval tool bound to this agent (retrieve_culinary_context). Use it when helpful; otherwise reason from your knowledge of ingredients, techniques, and named dishes.
"""},
    {"role": "human", "content": """
Task:
Return ONLY a single line in the exact format: [Country], [Region]
- Country: full official name (e.g., "United States", not "USA").
- Region: a specific intra-country region (e.g., North, South, East, West, Central, Northeast, etc.).
- Use "All" only if the dish is truly nationwide.
- If no region is identifiable, write "Unknown".
- Do NOT include any explanation before or after the bracketed answer.

Cues to consider:
- Ingredients (grains, staple flours, spice blends), cooking methods, named dishes, iconic sides.
- Example mappings:
  * Brazil: pão de queijo / tapioca flour / queijo minas → [Brazil], [South]
  * Japan: sushi / rice vinegar / tempura → [Japan], [All]
  * India: dosa / idli / coconut → [India], [South]; naan / paneer / tandoor → [India], [North]
  * Ethiopia: teff / injera / wat / berbere → [Ethiopia], [Unknown]
  * China: dim sum → [China], [South]; hot pot (Sichuan/Chongqing-style) → [China], [West]
  * Thailand: khao soi → [Thailand], [North]; som tam → [Thailand], [Northeast]

Few-shot examples:
Input: "Fermented teff flatbread served with spicy stews (wat) and berbere."
Output: [Ethiopia], [Unknown]

Input: "Cheese bread made with tapioca starch, typical with churrasco in the south."
Output: [Brazil], [South]

Input: "Batter of rice and urad dal, steamed into soft cakes, served with coconut chutney."
Output: [India], [South]

Input: "Assorted small bites with tea in bamboo steamers, hallmark of Cantonese cuisine."
Output: [China], [South]

Now produce ONLY the answer for the current input as [Country], [Region].
"""},
]

culinary_detective = create_agent(model=llm, tools=[retrieve_culinary_context])

In [15]:
result = culinary_detective.invoke({
    "messages": culinary_detective_messages + [
        {"role": "human", "content": "The dish is spicy, coconut-based, and served with rice."}
    ]
})
print(result)

{'messages': [SystemMessage(content='\nYou are an expert culinary anthropologist. From ingredients and a short description, identify the country and the SPECIFIC region within that country where the dish is most associated.\n\nYou can consult a retrieval tool bound to this agent (retrieve_culinary_context). Use it when helpful; otherwise reason from your knowledge of ingredients, techniques, and named dishes.\n', additional_kwargs={}, response_metadata={}, id='ef15957c-ed1e-43dd-b98d-50270cde169a'), HumanMessage(content='\nTask:\nReturn ONLY a single line in the exact format: [Country], [Region]\n- Country: full official name (e.g., "United States", not "USA").\n- Region: a specific intra-country region (e.g., North, South, East, West, Central, Northeast, etc.).\n- Use "All" only if the dish is truly nationwide.\n- If no region is identifiable, write "Unknown".\n- Do NOT include any explanation before or after the bracketed answer.\n\nCues to consider:\n- Ingredients (grains, staple fl

In [16]:
def extract_culinary_detective_answer(response: str) -> tuple[str, str]:
    # TODO: Extract the country and region from the response
    # return country, region
    import re
    
    # Look for pattern [country], [region] or variations
    bracket_pattern = r'\[([^\]]+)\]\s*,\s*\[([^\]]+)\]'
    match = re.search(bracket_pattern, response)
    
    if match:
        country = match.group(1).strip()
        region = match.group(2).strip()
        return country, region
    
    # Fallback: look for two items separated by comma
    lines = response.strip().split('\n')
    for line in lines:
        # Skip lines that are too long (likely explanations)
        if len(line) > 100:
            continue
        # Look for comma-separated values
        if ',' in line:
            parts = line.split(',')
            if len(parts) >= 2:
                country = parts[0].strip().strip('[]"\'')
                region = parts[1].strip().strip('[]"\'')
                # Clean up common prefixes
                for prefix in ['The answer is', 'Answer:', 'Country:', 'Region:']:
                    country = country.replace(prefix, '').strip()
                    region = region.replace(prefix, '').strip()
                return country, region
    
    # Last resort: return empty strings
    return "", ""

In [17]:
# extract_culinary_detective_answer(result)

### 👄Lingua Locale

In [18]:
lingua_locale_messages = [
    {"role": "system", "content": """
You are an expert in languages, scripts, orthography, and regional vocabulary. Determine which country's website or text a sentence most likely comes from.
"""},
    {"role": "user", "content": """
Return ONLY a single line: [Country]
- Use the full official country name (e.g., "United States", not "USA").
- No explanations. Do not output anything else.

Heuristics:
- English: colour, organise, centre → [United Kingdom]; sidewalk, color, organize → [United States].
- Portuguese: autocarro, factura, telemóvel → [Portugal]; ônibus, nota fiscal, celular → [Brazil].
- Spanish: vos, colectivo, pileta → [Argentina]; ordenador → [Spain]; autobús → [Mexico/Spain], coche (Spain) vs carro (LatAm).
- Chinese: Traditional characters (臺灣、臺北、繁體) → [Taiwan]; Simplified (中国、广州、简体) → [China].
- Cyrillic specifics: дј / ђ / ћ / љ / њ → [Montenegro]; девојка / ђак more typical of [Serbia].
- French: anglicisms + CAD context → [Canada]; métropolitain cues → [France].
- Haitian Creole → [Haiti].

Examples:
Input: "Please, colour is the preferred spelling in our centre."
Output: [United Kingdom]

Input: "Clique para imprimir a fatura no seu telemóvel."
Output: [Portugal]

Input: "Синоћ сам видио дјевојку у Подгорици."
Output: [Montenegro]

Input: "這是臺灣本地的頁面。"
Output: [Taiwan]

Now produce ONLY [Country] for the current sentence.
"""},
]

lingua_locale = create_agent(model=llm, tools=[])

In [19]:
def extract_lingua_locale_answer(response: str) -> tuple[str, str]:
    # TODO: Extract the country and "none" from the response
    # only the first field is used, the second is a dummy field to make the return type consistent
    # return country, "none"
    import re
    
    # Look for pattern [country] in brackets
    bracket_pattern = r'\[([^\]]+)\]'
    match = re.search(bracket_pattern, response)
    
    if match:
        country = match.group(1).strip()
        return country, "none"
    
    # Fallback: look for country name in response
    lines = response.strip().split('\n')
    for line in lines:
        # Skip lines that are too long (likely explanations)
        if len(line) > 100:
            continue
        # Clean the line
        clean_line = line.strip().strip('[]"\'')
        # Remove common prefixes
        for prefix in ['The answer is', 'Answer:', 'Country:', 'The country is', 'This is from']:
            clean_line = clean_line.replace(prefix, '').strip()
        
        # If we have a short, cleaned line, it's likely the country
        if clean_line and len(clean_line) < 50:
            # Remove any trailing punctuation
            clean_line = clean_line.rstrip('.,;:')
            return clean_line, "none"
    
    # Last resort: return empty string
    return "", "none"

## Answering questions
This part includes how we load the questions and generate the prediction in desired format. 

In [20]:
def geoguesser(q: dict, print_raw_response=False) -> tuple[str, str]:
    if q["type"] == "GlobalTrekker":
        query = {"role": "user", "content": f"Paragraph: {q['paragraph']}"}
        messages, agent, extractor = global_trekker_messages, global_trekker, extract_global_trekker_answer
    elif q["type"] == "CulinaryDetective":
        query = {"role": "user", "content": f"Ingredients: {q['ingredient']}. Description: {q['description']}"}
        messages, agent, extractor = culinary_detective_messages, culinary_detective, extract_culinary_detective_answer
    else: #q["type"] == "LinguaLocale":
        query = {"role": "user", "content": f"Sentence: {q['sentence']}"}
        messages, agent, extractor = lingua_locale_messages, lingua_locale, extract_lingua_locale_answer

    response_all = agent.invoke({"messages": messages + [query]})
    response = response_all["messages"][-1].content
    if print_raw_response: print(f"{q['type']}: {response_all}")
    return extractor(response)

In [27]:
import json

# Here, we load the examples questions. Public/private set will be in the same format
dataset_name = "public.jsonl"
questions = []
with open(dataset_name, "r") as f:
    for line in f.readlines():
        questions.append(json.loads(line))

In [28]:
# Test run on one question
# You might want to save the raw response for debugging answer formatting/extraction
# If the extracted answer seems off, check the raw response instead of running inference repeatedly
geoguesser(questions[0], print_raw_response=True)

GlobalTrekker: {'messages': [SystemMessage(content='You are an expert in world geography and cultural geography. Given a descriptive paragraph, infer the most likely country and city with high precision.', additional_kwargs={}, response_metadata={}, id='16962dec-077c-4b5a-b261-c042a37adb4b'), HumanMessage(content='\nRead the paragraph and extract:\n- Country: use the full official country name (e.g., "United States", not "USA").\n- City: a specific city if clearly indicated by clues (landmarks, neighborhoods, transit lines, local foods, dialects). If not identifiable, write "Unknown".\n\nStrict output format (no explanations):\n[Country], [City]\n\nGuidelines:\n- Prefer unique cues (local transit names, street names, postal formats, phone codes, currency, cuisine, sports clubs).\n- If multiple cities fit, pick the single most likely one.\n- If the paragraph is generic or only country-level, return "Unknown" for city.\n- Do not include any extra text besides the bracketed pair.\n\nExamp

('Australia', 'Unknown')

In [29]:
# Sample script to generate answers
from tqdm import tqdm
answers = []
for q in tqdm(questions):
    try:
        country, category = geoguesser(q)
        answers.append(f"{q['type']}\t{country}\t{category}")
    except Exception as e:
        print(f"Error processing question {q}: {e}")
        answers.append(f"{q['type']}\tUnknown\tUnknown")

with open("public.txt", "w") as f:
    for answer in answers:
        f.write(answer + "\n")

print("Saved predictions to public.txt (private set)")

100%|██████████| 170/170 [24:45<00:00,  8.74s/it] 

Saved predictions to public.txt (private set)


## Evaluation
This is how we calculate the scores on Gradescope (details subject to change, but the general logic will stay the same).

In [30]:
def soft_match(answer, expectedAnswer):
    score = 0.0
    if expectedAnswer in answer:
        score = len(expectedAnswer) / len(answer)
    return score

def exact_match(answer, expectedAnswer):
    score = 0.0
    if expectedAnswer == answer:
        score = 1.0
    return score

In [31]:
answers = []
for q in questions:
    answers.append((q["type"], q["country"], q.get("city", q.get("region", ""))))
with open("public.txt", "r") as f:
    preds = [line.split("\t") for line in f.readlines()]

scores = {"GlobalTrekker": [], "CulinaryDetective": [], "LinguaLocale": []}
for (q_type, exp_country, exp_place), (p_type, pred_country, pred_place) in zip(answers, preds):
    assert q_type == p_type
    country_score = soft_match(pred_country, exp_country)
    category_score = 0.0
    weights = [0.0, 0.0]
    if q_type == "GlobalTrekker":
        #  correct country -> 80%, correct country and city -> +20%
        weights = [0.8, 0.2]
        if country_score > 0:
            if exp_place == "None": category_score = 1.0
            else: category_score = soft_match(pred_place, exp_place)
    elif q_type == "CulinaryDetective":
        # correct country -> 60%, correct country and region -> +40%
        weights = [0.6, 0.4]
        if country_score > 0:
            if exp_place == "None": category_score = 1.0
            else: category_score = exact_match(pred_place, exp_place)
    else: # LinguaLocale
        # correct country -> 60%, matched official language -> +40%
        weights = [0.6, 0.4]
        if country_score > 0:
            category_score = 1.0
        else: # incorrect country. language match works only if pred_country is a clean answer
            exp_langs = list_of_countries.get(exp_country, [])
            pred_langs = list_of_countries.get(pred_country, [])
            if any(lang in exp_langs for lang in pred_langs):
                category_score = 1.0

    score = weights[0] * country_score + weights[1] * category_score
    scores[q_type].append(score)

for q_type, score_list in scores.items():
    avg_score = sum(score_list) / len(score_list)
    print(f"{q_type} Average Score: {avg_score:.4f}")

GlobalTrekker Average Score: 0.8195
CulinaryDetective Average Score: 0.5273
LinguaLocale Average Score: 0.8778
